In [3]:
import tkinter as tk
from tkinter import ttk
import math

class Calculator:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Python Calculator")
        self.window.geometry("300x400")
        self.window.resizable(False, False)
        
        # Variables
        self.current = ""
        self.total = 0
        self.input_value = True
        self.result = False
        
        # Create display
        self.display_var = tk.StringVar()
        self.display_var.set("0")
        
        # Create GUI elements
        self.create_display()
        self.create_buttons()
        
    def create_display(self):
        """Create the calculator display"""
        display_frame = tk.Frame(self.window)
        display_frame.pack(expand=True, fill="both")
        
        self.display = tk.Entry(
            display_frame,
            textvariable=self.display_var,
            font=("Arial", 16),
            bd=10,
            insertwidth=2,
            width=14,
            justify="right",
            state="readonly"
        )
        self.display.pack(expand=True, fill="both", padx=5, pady=5)
        
    def create_buttons(self):
        """Create calculator buttons"""
        button_frame = tk.Frame(self.window)
        button_frame.pack(expand=True, fill="both")
        
        # Button layout
        buttons = [
            ['C', '±', '%', '÷'],
            ['7', '8', '9', '×'],
            ['4', '5', '6', '-'],
            ['1', '2', '3', '+'],
            ['0', '.', '=']
        ]
        
        for i, row in enumerate(buttons):
            for j, btn_text in enumerate(row):
                if btn_text == '0':
                    # Make 0 button span 2 columns
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j, columnspan=2, sticky="nsew", padx=1, pady=1)
                elif btn_text == '=':
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        bg="orange",
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j+1, sticky="nsew", padx=1, pady=1)
                else:
                    # Regular button
                    bg_color = "lightgray" if btn_text in ['C', '±', '%'] else "white"
                    if btn_text in ['÷', '×', '-', '+']:
                        bg_color = "orange"
                    
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        bg=bg_color,
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j, sticky="nsew", padx=1, pady=1)
        
        # Configure grid weights
        for i in range(5):
            button_frame.grid_rowconfigure(i, weight=1)
        for j in range(4):
            button_frame.grid_columnconfigure(j, weight=1)
    
    def button_click(self, char):
        """Handle button clicks"""
        if self.result and char.isdigit():
            self.current = ""
            self.result = False
            
        if char.isdigit() or char == ".":
            self.number_press(char)
        elif char in "+-×÷":
            self.operation_press(char)
        elif char == "=":
            self.equals_press()
        elif char == "C":
            self.clear_all()
        elif char == "±":
            self.plus_minus()
        elif char == "%":
            self.percentage()
    
    def number_press(self, num):
        """Handle number button presses"""
        if self.input_value:
            self.current += str(num)
            self.display_var.set(self.current)
        else:
            self.current = str(num)
            self.display_var.set(self.current)
            self.input_value = True
    
    def operation_press(self, op):
        """Handle operation button presses"""
        if self.current:
            if not self.input_value:
                self.current = str(self.total)
            else:
                try:
                    self.total = float(self.current)
                except ValueError:
                    self.display_var.set("Error")
                    return
            
            self.current = ""
            self.input_value = False
            self.operation = op
            self.display_var.set(str(self.total))
    
    def equals_press(self):
        """Handle equals button press"""
        if not self.current or not hasattr(self, 'operation'):
            return
            
        try:
            if self.operation == "+":
                self.total += float(self.current)
            elif self.operation == "-":
                self.total -= float(self.current)
            elif self.operation == "×":
                self.total *= float(self.current)
            elif self.operation == "÷":
                if float(self.current) == 0:
                    self.display_var.set("Error")
                    return
                self.total /= float(self.current)
            
            # Format result
            if self.total == int(self.total):
                self.display_var.set(str(int(self.total)))
            else:
                self.display_var.set(str(self.total))
                
            self.current = str(self.total)
            self.input_value = False
            self.result = True
            
        except (ValueError, ZeroDivisionError):
            self.display_var.set("Error")
            self.clear_all()
    
    def clear_all(self):
        """Clear all values"""
        self.current = ""
        self.total = 0
        self.input_value = True
        self.result = False
        self.display_var.set("0")
        if hasattr(self, 'operation'):
            del self.operation
    
    def plus_minus(self):
        """Toggle positive/negative"""
        if self.current:
            if self.current[0] == '-':
                self.current = self.current[1:]
            else:
                self.current = '-' + self.current
            self.display_var.set(self.current)
    
    def percentage(self):
        """Calculate percentage"""
        if self.current:
            try:
                result = float(self.current) / 100
                self.current = str(result)
                self.display_var.set(self.current)
            except ValueError:
                self.display_var.set("Error")
    
    def run(self):
        """Start the calculator"""
        self.window.mainloop()


class CommandLineCalculator:
    """Simple command-line calculator"""
    
    @staticmethod
    def evaluate(expression):
        """Safely evaluate mathematical expressions"""
        # Replace common symbols
        expression = expression.replace('×', '*').replace('÷', '/')
        
        try:
            # Only allow safe characters
            allowed_chars = set('0123456789+-*/.() ')
            if not all(c in allowed_chars for c in expression):
                return "Error: Invalid characters"
            
            result = eval(expression)
            return result if result != int(result) else int(result)
        except ZeroDivisionError:
            return "Error: Division by zero"
        except Exception:
            return "Error: Invalid expression"
    
    @staticmethod
    def run():
        """Run command-line calculator"""
        print("Command Line Calculator")
        print("Enter expressions like: 2 + 3, 10 * 5, etc.")
        print("Use 'quit' or 'exit' to close")
        print("-" * 30)
        
        while True:
            try:
                expression = input(">>> ").strip()
                
                if expression.lower() in ['quit', 'exit', 'q']:
                    print("Goodbye!")
                    break
                
                if not expression:
                    continue
                    
                result = CommandLineCalculator.evaluate(expression)
                print(f"= {result}")
                
            except KeyboardInterrupt:
                print("\nGoodbye!")
                break
            except EOFError:
                print("\nGoodbye!")
                break


def main():
    """Main function to choose calculator mode"""
    print("Python Calculator")
    print("1. GUI Calculator")
    print("2. Command Line Calculator")
    
    try:
        choice = input("Choose mode (1 or 2): ").strip()
        
        if choice == "1":
            calc = Calculator()
            calc.run()
        elif choice == "2":
            CommandLineCalculator.run()
        else:
            print("Invalid choice. Starting GUI calculator...")
            calc = Calculator()
            calc.run()
            
    except KeyboardInterrupt:
        print("\nGoodbye!")


if __name__ == "__main__":
    main()

Python Calculator
1. GUI Calculator
2. Command Line Calculator


Choose mode (1 or 2):  1


In [5]:
import tkinter as tk
from tkinter import ttk
import math

class Calculator:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Python Calculator")
        self.window.geometry("300x400")
        self.window.resizable(False, False)
        
        # Variables
        self.current = ""
        self.total = 0
        self.input_value = True
        self.result = False
        
        # Create display
        self.display_var = tk.StringVar()
        self.display_var.set("0")
        
        # Create GUI elements
        self.create_display()
        self.create_buttons()
        
    def create_display(self):
        """Create the calculator display"""
        display_frame = tk.Frame(self.window)
        display_frame.pack(expand=True, fill="both")
        
        self.display = tk.Entry(
            display_frame,
            textvariable=self.display_var,
            font=("Arial", 16),
            bd=10,
            insertwidth=2,
            width=14,
            justify="right",
            state="readonly"
        )
        self.display.pack(expand=True, fill="both", padx=5, pady=5)
        
    def create_buttons(self):
        """Create calculator buttons"""
        button_frame = tk.Frame(self.window)
        button_frame.pack(expand=True, fill="both")
        
        # Button layout
        buttons = [
            ['C', '±', '%', '÷'],
            ['7', '8', '9', '×'],
            ['4', '5', '6', '-'],
            ['1', '2', '3', '+'],
            ['0', '.', '=']
        ]
        
        for i, row in enumerate(buttons):
            for j, btn_text in enumerate(row):
                if btn_text == '0':
                    # Make 0 button span 2 columns
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j, columnspan=2, sticky="nsew", padx=1, pady=1)
                elif btn_text == '=':
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        bg="orange",
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j+1, sticky="nsew", padx=1, pady=1)
                else:
                    # Regular button
                    bg_color = "lightgray" if btn_text in ['C', '±', '%'] else "white"
                    if btn_text in ['÷', '×', '-', '+']:
                        bg_color = "orange"
                    
                    btn = tk.Button(
                        button_frame,
                        text=btn_text,
                        font=("Arial", 14),
                        bg=bg_color,
                        command=lambda t=btn_text: self.button_click(t)
                    )
                    btn.grid(row=i, column=j, sticky="nsew", padx=1, pady=1)
        
        # Configure grid weights
        for i in range(5):
            button_frame.grid_rowconfigure(i, weight=1)
        for j in range(4):
            button_frame.grid_columnconfigure(j, weight=1)
    
    def button_click(self, char):
        """Handle button clicks"""
        if self.result and char.isdigit():
            self.current = ""
            self.result = False
            
        if char.isdigit() or char == ".":
            self.number_press(char)
        elif char in "+-×÷":
            self.operation_press(char)
        elif char == "=":
            self.equals_press()
        elif char == "C":
            self.clear_all()
        elif char == "±":
            self.plus_minus()
        elif char == "%":
            self.percentage()
    
    def number_press(self, num):
        """Handle number button presses"""
        if self.input_value:
            self.current += str(num)
            self.display_var.set(self.current)
        else:
            self.current = str(num)
            self.display_var.set(self.current)
            self.input_value = True
    
    def operation_press(self, op):
        """Handle operation button presses"""
        if self.current:
            if not self.input_value:
                self.current = str(self.total)
            else:
                try:
                    self.total = float(self.current)
                except ValueError:
                    self.display_var.set("Error")
                    return
            
            self.current = ""
            self.input_value = False
            self.operation = op
            self.display_var.set(str(self.total))
    
    def equals_press(self):
        """Handle equals button press"""
        if not self.current or not hasattr(self, 'operation'):
            return
            
        try:
            if self.operation == "+":
                self.total += float(self.current)
            elif self.operation == "-":
                self.total -= float(self.current)
            elif self.operation == "×":
                self.total *= float(self.current)
            elif self.operation == "÷":
                if float(self.current) == 0:
                    self.display_var.set("Error")
                    return
                self.total /= float(self.current)
            
            # Format result
            if self.total == int(self.total):
                self.display_var.set(str(int(self.total)))
            else:
                self.display_var.set(str(self.total))
                
            self.current = str(self.total)
            self.input_value = False
            self.result = True
            
        except (ValueError, ZeroDivisionError):
            self.display_var.set("Error")
            self.clear_all()
    
    def clear_all(self):
        """Clear all values"""
        self.current = ""
        self.total = 0
        self.input_value = True
        self.result = False
        self.display_var.set("0")
        if hasattr(self, 'operation'):
            del self.operation
    
    def plus_minus(self):
        """Toggle positive/negative"""
        if self.current:
            if self.current[0] == '-':
                self.current = self.current[1:]
            else:
                self.current = '-' + self.current
            self.display_var.set(self.current)
    
    def percentage(self):
        """Calculate percentage"""
        if self.current:
            try:
                result = float(self.current) / 100
                self.current = str(result)
                self.display_var.set(self.current)
            except ValueError:
                self.display_var.set("Error")
    
    def run(self):
        """Start the calculator"""
        self.window.mainloop()


class CommandLineCalculator:
    """Simple command-line calculator"""
    
    @staticmethod
    def evaluate(expression):
        """Safely evaluate mathematical expressions"""
        # Replace common symbols
        expression = expression.replace('×', '*').replace('÷', '/')
        
        try:
            # Only allow safe characters
            allowed_chars = set('0123456789+-*/.() ')
            if not all(c in allowed_chars for c in expression):
                return "Error: Invalid characters"
            
            result = eval(expression)
            return result if result != int(result) else int(result)
        except ZeroDivisionError:
            return "Error: Division by zero"
        except Exception:
            return "Error: Invalid expression"
    
    @staticmethod
    def run():
        """Run command-line calculator"""
        print("Command Line Calculator")
        print("Enter expressions like: 2 + 3, 10 * 5, etc.")
        print("Use 'quit' or 'exit' to close")
        print("-" * 30)
        
        while True:
            try:
                expression = input(">>> ").strip()
                
                if expression.lower() in ['quit', 'exit', 'q']:
                    print("Goodbye!")
                    break
                
                if not expression:
                    continue
                    
                result = CommandLineCalculator.evaluate(expression)
                print(f"= {result}")
                
            except KeyboardInterrupt:
                print("\nGoodbye!")
                break
            except EOFError:
                print("\nGoodbye!")
                break


def main():
    """Main function to choose calculator mode"""
    print("Python Calculator")
    print("1. GUI Calculator")
    print("2. Command Line Calculator")
    
    try:
        choice = input("Choose mode (1 or 2): ").strip()
        
        if choice == "1":
            calc = Calculator()
            calc.run()
        elif choice == "2":
            CommandLineCalculator.run()
        else:
            print("Invalid choice. Starting GUI calculator...")
            calc = Calculator()
            calc.run()
            
    except KeyboardInterrupt:
        print("\nGoodbye!")


if __name__ == "__main__":
    main()

Python Calculator
1. GUI Calculator
2. Command Line Calculator


Choose mode (1 or 2):  2


Command Line Calculator
Enter expressions like: 2 + 3, 10 * 5, etc.
Use 'quit' or 'exit' to close
------------------------------


>>>  49+523-67*2


= 438


...

= 55


>>>  60+90


= 150


>>>  exit


Goodbye!
